1. What is this code actually doing?

In [1]:
def create_sliding_windows(data, K):
    X, y = [], []
    for i in range(len(data) - K):
        X.append(data[i:i+K])
        y.append(data[i+K])

    X = np.array(X)  # Shape: (n_samples, K)
    y = np.array(y)  # Shape: (n_samples,)
    
    # Convert to PyTorch tensors
    X_tensor = torch.from_numpy(X)
    y_tensor = torch.from_numpy(y).unsqueeze(1)  

    return X_tensor, y_tensor

2. In this vanilla RNN model:
    - What is input_size, hidden_size, and output_size
    - What is ho, created in line 9?
    - Which value appears in out[:, -1, :]

In [ ]:
# Vanilla RNN Model
class VanillaRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, output_size=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        return self.fc(out[:, -1, :])

3. Explain the result shapes

In [ ]:
X, y = next(iter(train_loader))
X.shape, model(X).shape
# Result: (torch.Size([512, 20, 1]), torch.Size([512, 1]))

4. What is this function actually doing?
    - What is the torch.cat in line 12 actually doing?

In [ ]:
# Prediction function
def predict_next_values(model, initial_seq, n_steps):
    model.eval()
    predictions = []
    current_seq = initial_seq.clone()
    
    with torch.no_grad():
        for _ in range(n_steps):
            pred = model(current_seq).item()
            predictions.append(pred)
            # Update sequence: remove oldest, add new prediction
            current_seq = torch.cat([current_seq[:, 1:, :], 
                                  torch.tensor([[[pred]]]).float()], dim=1)
    return np.array(predictions)

5. In this code the self.hidden is initializad in the __init__ instead of in the forward method. Why?

In [ ]:
class LSTM(nn.Module):
    
    def __init__(self,input_size = 1, hidden_size = 50, out_size = 1):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size,out_size)
        self.hidden = (torch.zeros(1,1,hidden_size),torch.zeros(1,1,hidden_size))
    
    def forward(self,seq):
        lstm_out, self.hidden = self.lstm(seq.view(len(seq),1,-1), self.hidden)
        pred = self.linear(lstm_out.view(len(seq),-1))
        return pred[-1]

6. This dataloader will be using from training al LSTM neural network.
    - Explain all the components of the X tensor, given the printed shape

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)

X, y = next(iter(train_loader))
print(X.shape)
# printed: torch.Size([3, 7, 2])

7. In the forward pass of the LSTM network, two tensors h0 and c0 are created.
    - What are these vectors?
    - What is the component of the resultant value of the lstm used of the linear layer?

In [ ]:
def forward(self, x):
    batch_size = x.shape[0]
    h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
    c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()

    _, (hn, _) = self.lstm(x, (h0, c0))
    out = self.linear(hn[0]).flatten()  

    return out

8. In the example of using a RNN network for classifying digit images.
    - Why is the input_size equal to 28
    - What is the rationale of doing so?

In [ ]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size=28, hidden_size=128, num_classes=10):
        super().__init__()
        self.rnn = nn.RNN(
            input_size=input_size,  # 28 pixels per row
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True  # Input shape: [batch, seq_len, input_size]
        )
        self.fc = nn.Linear(hidden_size, num_classes)  # Output layer